# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark RDD - SOLUTION
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://images.datacamp.com/image/upload/v1676303379/Marketing/Blog/PySpark_RDD_Cheat_Sheet.pdf) is useful.  As is, the [RDD API reference](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html).

In [1]:
from pyspark import SparkContext, SparkConf
import numpy as np
import operator

In [2]:
conf=SparkConf().setAppName("Lab4-rdd").setMaster("local[*]")
sc = SparkContext(conf=conf)

Using PySpark and RDD's on the https://coding.csel.io machines is slow -- most of the code is executed in Python and this is much less efficient than the java-based code using the PySpark dataframes. Be patient and trying using `.cache()` to cache the output of joins. You may want to start with a reduced set of data before running the full task. You can use the `sample()` method to extract just a sample of the data or use 

These two RDD's are called "rawCitations" and "rawPatents" because you probably want to process them futher (e.g. convert them to integer types, etc). 

The `textFile` function returns data in strings. This should work fine for this lab.

Other methods you use might return data in type `Byte`. If you haven't used Python `Byte` types before, google it. You can convert a value of `x` type byte into e.g. a UTF8 string using `x.decode('uft-8')`. Alternatively, you can use the `open` method of the gzip library to read in all the lines as UTF-8 strings like this:
```
import gzip
with gzip.open('cite75_99.txt.gz', 'rt',encoding='utf-8') as f:
    rddCitations = sc.parallelize( f.readlines() )
```
This is less efficient than using `textFile` because `textFile` would use the underlying HDFS or other file system to read the file across all the worker nodes while the using `gzip.open()...readlines()` will read all the data in the frontend and then distribute it to all the worker nodes.

In [3]:
rddCitations = sc.textFile("cite75_99.txt.gz")
rddPatents = sc.textFile("apat63_99.txt.gz")

The data looks like the following.

In [4]:
rddCitations.take(5)

['"CITING","CITED"',
 '3858241,956203',
 '3858241,1324234',
 '3858241,3398406',
 '3858241,3557384']

In [5]:
rddPatents.take(5)

['"PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD"',
 '3070801,1963,1096,,"BE","",,1,,269,6,69,,1,,0,,,,,,,',
 '3070802,1963,1096,,"US","TX",,1,,2,6,63,,0,,,,,,,,,',
 '3070803,1963,1096,,"US","IL",,1,,2,6,63,,9,,0.3704,,,,,,,',
 '3070804,1963,1096,,"US","OH",,1,,2,6,63,,3,,0.6667,,,,,,,']

In other words, they are a single string with multiple CSV's. You will need to convert these to (K,V) pairs, probably convert the keys to `int` and so on. You'll need to `filter` out the header string as well since there's no easy way to extract all the lines except the first.

# Solution:


Function to process and extract relevant fields from patent and citation data, allowing us to work with the patent ID and state in the patents dataset, and the citing and cited patent IDs in the citations dataset.

In [6]:
# Function to split patent data by commas and extract relevant fields
def extractPatentFields(record):
    fields = record.split(',')
    return (fields[0], fields[5])

# Function to split citation data by commas and extract relevant fields
def extractCitationFields(record):
    fields = record.split(',')
    return (fields[0], fields[1].strip())



In [7]:
# Mapping the RDDs for patents and citations
patentsRDD = rddPatents.map(extractPatentFields)
citationsRDD = rddCitations.map(extractCitationFields)



In [8]:
# Take a look at the first few records of both datasets
print(patentsRDD.take(5))
print(citationsRDD.take(5))



[('"PATENT"', '"POSTATE"'), ('3070801', '""'), ('3070802', '"TX"'), ('3070803', '"IL"'), ('3070804', '"OH"')]
[('"CITING"', '"CITED"'), ('3858241', '956203'), ('3858241', '1324234'), ('3858241', '3398406'), ('3858241', '3557384')]


## Joining Patents table with Citations table


Joining `CITING`,`CITED`,`CITING_POSTATE` 

In [9]:
# Joining citations and patents data and caching the result
joinStep = citationsRDD.join(patentsRDD).cache()
print(joinStep.take(5))



[('3862708', ('2387598', '"PA"')), ('3862708', ('2630834', '"PA"')), ('3862708', ('3656662', '"PA"')), ('3870303', ('1934381', '"OH"')), ('3870303', ('2574363', '"OH"'))]


In [10]:
# Function to swap the data for further processing
def swapFields(entry):
    citing_patent, (cited_patent, citing_state) = entry
    return (cited_patent, (citing_patent, citing_state))

# Function to map the joined data
def mapCitedData(entry):
    cited_patent, ((citing_patent, citing_state), cited_state) = entry
    return (citing_patent, (citing_state, cited_patent, cited_state))



Here, we transforms the data so that the `CITED` becomes the key and then joins it with patentsRDD to retrieve the `CITED_POSTATE`. 

In [11]:
# Performing transformation and join with patents data
stateDataRDD = joinStep.map(swapFields)
joinedStep = stateDataRDD.join(patentsRDD).cache()
mappedData = joinedStep.map(mapCitedData)



The `stateMatchFilter` function filters records where both the `CITING_STATE` and `CITED_STATE` patents have non-empty states that match. The `countCitations` function counts how many times each citing patent appears by returning a tuple of the citing patent and the count 1.

In [12]:
# Filter function to retain only records where both states match and are non-empty
def stateMatchFilter(entry):
    citing_patent, (citing_state, cited_patent, cited_state) = entry
    return citing_state != '""' and cited_state != '""' and citing_state == cited_state

# Counter function to track citing patents
def countCitations(entry):
    citing_patent, (citing_state, cited_patent, cited_state) = entry
    return (citing_patent, 1)



The code below filters the data to retain only matching state records, counts the number of citations for each citing patent, and then sorts the results by citation count in descending order. The final result gives the most frequently cited patents.

In [13]:
# Filtering, counting, and sorting citations by count
filteredData = mappedData.filter(stateMatchFilter)
citationCounts = filteredData.map(countCitations).reduceByKey(lambda x, y: x + y)
sortedCitations = citationCounts.sortBy(lambda entry: entry[1], ascending=False)



In [14]:
# Display top 10 citations
print(sortedCitations.take(10))



[('5959466', 125), ('5983822', 103), ('6008204', 100), ('5952345', 98), ('5998655', 96), ('5958954', 96), ('5936426', 94), ('5913855', 90), ('5739256', 90), ('5978329', 90)]


In the above column, we can see the patent ID na dthe number of same state citation

The `createPatentKeyValue` function splits each patent record into a key-value pair, where the patent ID is the key and the remaining data is the value. The `handleMissingCount` function checks for patents with missing citation counts and assigns a default count of 0 if none is found.

In [15]:
# Function to transform patent data into key-value pairs
def createPatentKeyValue(record):
    split_line = record.split(',')
    return (split_line[0], ",".join(split_line[1:]))

# Function to handle missing citation counts (i.e., assign 0 if none found)
def handleMissingCount(record):
    patent_id, (patent_details, citation_count) = record
    if citation_count is None:
        citation_count = 0
    return (patent_id, (patent_details, citation_count))





This code performs a left outer join with the citation counts to ensure all patents are included, even those without citations. It then processes the joined data to handle any missing citation counts by assigning a default value of 0 where necessary.

In [16]:
# Left outer join patents with citation counts and process missing counts
patentKeyValueRDD = rddPatents.map(createPatentKeyValue)
joinedPatentCitation = patentKeyValueRDD.leftOuterJoin(sortedCitations).cache()
finalDataRDD = joinedPatentCitation.map(handleMissingCount)



In [17]:
# Take a look at the final data
print(finalDataRDD.take(10))



[('3070839', ('1963,1096,,"US","TN",,2,,425,5,51,,8,,0.4063,,,,,,,', 0)), ('3071252', ('1963,1096,,"CA","",,1,,211,6,65,,5,,0.56,,,,,,,', 0)), ('3071953', ('1963,1103,,"US","PA",,2,,68,6,63,,2,,0,,,,,,,', 0)), ('3072623', ('1963,1103,,"US","DE",,2,,526,1,15,,0,,,,,,,,,', 0)), ('3072816', ('1963,1103,,"CH","",,3,,313,4,42,,0,,,,,,,,,', 0)), ('3073404', ('1963,1110,,"US","CA",,1,,180,5,55,,0,,,,,,,,,', 0)), ('3073761', ('1963,1110,,"US","NJ",,1,,205,1,19,,5,,0.72,,,,,,,', 0)), ('3074068', ('1963,1110,,"US","MI",,1,,134,6,69,,1,,0,,,,,,,', 0)), ('3074185', ('1963,1117,,"DE","",,1,,36,6,63,,1,,0,,,,,,,', 0)), ('3074244', ('1963,1117,,"US","NJ",,2,,62,6,69,,2,,0,,,,,,,', 0))]


The `sortByCitationCount` function extracts the citation count from each record, which consists of a patent ID and its associated details. This function is used to facilitate sorting the data by citation count in descending order.

In [18]:
# Function to sort the final data by citation count
def sortByCitationCount(record):
    patent_id, (patent_details, citation_count) = record
    return citation_count



In [19]:
# Sort the final output by citation count in descending order
finalSortedOutput = finalDataRDD.sortBy(lambda record: sortByCitationCount(record), ascending=False)

# Take the top 10 sorted records
print(finalSortedOutput.take(10))

[('5959466', ('1999,14515,1997,"US","CA",5310,2,,326,4,46,159,0,1,,0.6186,,4.8868,0.0455,0.044,,', 125)), ('5983822', ('1999,14564,1998,"US","TX",569900,2,,114,5,55,200,0,0.995,,0.7201,,12.45,0,0,,', 103)), ('6008204', ('1999,14606,1998,"US","CA",749584,2,,514,3,31,121,0,1,,0.7415,,5,0.0085,0.0083,,', 100)), ('5952345', ('1999,14501,1997,"US","CA",749584,2,,514,3,31,118,0,1,,0.7442,,5.1102,0,0,,', 98)), ('5958954', ('1999,14515,1997,"US","CA",749584,2,,514,3,31,116,0,1,,0.7397,,5.181,0,0,,', 96)), ('5998655', ('1999,14585,1998,"US","CA",,1,,560,1,14,114,0,1,,0.7387,,5.1667,,,,', 96)), ('5936426', ('1999,14466,1997,"US","CA",5310,2,,326,4,46,178,0,1,,0.58,,11.2303,0.0765,0.073,,', 94)), ('5739256', ('1998,13983,1995,"US","CA",70060,2,15,528,1,15,453,0,1,,0.8232,,15.1104,0.1124,0.1082,,', 90)), ('5925042', ('1999,14445,1997,"US","CA",733846,2,,606,3,32,242,0,1,,0.7382,,8.3471,0,0,,', 90)), ('5913855', ('1999,14417,1997,"US","CA",733846,2,,606,3,32,242,0,1,,0.7403,,8.3595,0,0,,', 90))]


In [20]:
# Sort the final output by citation count in descending order
finalSortedOutput = finalDataRDD.sortBy(lambda record: sortByCitationCount(record), ascending=False)

# Take the top 10 sorted records
top10Records = finalSortedOutput.take(10)

# Display each record line by line
for record in top10Records:
    print(record)


('5959466', ('1999,14515,1997,"US","CA",5310,2,,326,4,46,159,0,1,,0.6186,,4.8868,0.0455,0.044,,', 125))
('5983822', ('1999,14564,1998,"US","TX",569900,2,,114,5,55,200,0,0.995,,0.7201,,12.45,0,0,,', 103))
('6008204', ('1999,14606,1998,"US","CA",749584,2,,514,3,31,121,0,1,,0.7415,,5,0.0085,0.0083,,', 100))
('5952345', ('1999,14501,1997,"US","CA",749584,2,,514,3,31,118,0,1,,0.7442,,5.1102,0,0,,', 98))
('5958954', ('1999,14515,1997,"US","CA",749584,2,,514,3,31,116,0,1,,0.7397,,5.181,0,0,,', 96))
('5998655', ('1999,14585,1998,"US","CA",,1,,560,1,14,114,0,1,,0.7387,,5.1667,,,,', 96))
('5936426', ('1999,14466,1997,"US","CA",5310,2,,326,4,46,178,0,1,,0.58,,11.2303,0.0765,0.073,,', 94))
('5739256', ('1998,13983,1995,"US","CA",70060,2,15,528,1,15,453,0,1,,0.8232,,15.1104,0.1124,0.1082,,', 90))
('5925042', ('1999,14445,1997,"US","CA",733846,2,,606,3,32,242,0,1,,0.7382,,8.3471,0,0,,', 90))
('5913855', ('1999,14417,1997,"US","CA",733846,2,,606,3,32,242,0,1,,0.7403,,8.3595,0,0,,', 90))
